## Measuring bias significance
### Notebook for bootstrapping evaluation of evaluated dataset with computed heuristics

### Imports

In [1]:
!pip install transformers datasets

In [2]:
import transformers
from datasets import load_metric
import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

/home/luki/anaconda3/envs/env-QA/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### Parameters for number of iterations and number of selected items

In [3]:
num_samples = 100
sample_size = 800
num_for_average_metrics = 1

In [4]:
metric = load_metric("squad")

In [4]:
def compute_metrics_for_sample(sample):
    """Computation of metrics for dataset sample
    Computes exact match and F1 between predicted and ground truth answers

    Args:
        sample (Dataframe): sample from dataset

    Returns:
        dict: computed metrics
    """
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in zip(sample['id'], sample['prediction_text'])]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in sample]
    return metric.compute(predictions=formatted_predictions, references=references)

In [5]:
# variable for extensive logging, if True, the logging will be enabled, if False, only one logging file is used
write_to_log_files = False 

In [6]:
# name of the dataset
dataset = 'squad_base' 

### Data loading

In [7]:
# loading of dataset into pandas Dataframe from .json format
data = pd.read_json('valid_pred_labeled_with_added_from_func.json')

### Creation of logging files 
#### comment out after the first run

In [10]:
with open("metrics_for_specific_runs.csv", "w") as file:
    file.write(f"name,samples,sample_size,iters,field,threshold,type,exact_match_quantile_0.025,exact_match_quantile_0.975,exact_match_mean,f1_quantile_0.025,f1_quantile_0.975,f1_mean,len_lower,len_higher")

In [11]:
with open("average_metrics_for_runs.csv", "w") as file:
    file.write(f"name,samples,sample_size,iters,field,threshold,type,exact_match_quantile_0.025,exact_match_quantile_0.975,exact_match_mean,f1_quantile_0.025,f1_quantile_0.975,f1_mean,len_lower,len_higher")

In [12]:
with open("evaluated_metrics_for_average.csv", "w") as file:
    file.write(f"metric,samples,sample_size,iters,field,threshold,is_not_overlap,distance,len_lower,len_higher")

In [13]:
with open("metrics_with_intervals_distances_for_dataset_comparison.csv", "w") as file:
    file.write(f"dataset,field,threshold,lower_interval_em,distance,higher_interval_em,lower_interval_f1,distance,higher_interval_f1,samples,sample_size,len_lower,len_higher")

In [14]:
with open("data_for_violin.csv", "w") as file:
    file.write(f"dataset\tfield\tthreshold\tlower_interval_em\tdistance\thigher_interval_em\tlower_interval_f1\tdistance\thigher_interval_f1\tsamples\tsample_size\tlen_lower\tlen_higher\tlower_em_list\thigher_em_list\tlower_f1_list\thigher_f1_list")

### Computation of metrics for samples

In [15]:
def compute_metrics_for_bunch(data):
    """Sampling the dataset for specified number of iterations and computation of metrics for samples

    Args:
        data (Pandas Dataframe): dataset with computed heuristics

    Returns:
        Pandas Dataframe: computed heuristics for samples
    """
    exact_list = []
    f1_list = []

    for i in tqdm(range(num_samples)):
        df = data.sample(n=sample_size)
        sample = Dataset.from_pandas(df)
        metrics1 = compute_metrics_for_sample(sample)
        exact_list.append(metrics1['exact_match'])
        f1_list.append(metrics1['f1'])
    
    d = {'exact_match': exact_list, 'f1': f1_list}
    df = pd.DataFrame(d)

    return df

In [16]:
def find_the_distance_between_intervals(lower_025, lower_975, higher_025, higher_975):
    """Detect if there is some interval overlap between the two intervals or if there is not
    If there is not an overlap, it computes the distance between 2.5% and 97.5% quantiles

    Args:
        lower_025 (int): 2.5% quantile for lower subset
        lower_975 (int): 97.5% quantile for higher subset
        higher_025 (int): 2.5% quantile for lower subset
        higher_975 (int): 97.5% quantile for higher subset

    Returns:
        bool, decimal: boolean flag and the distance between intervals
    """
    distance_between_intervals = 0
    if lower_975 > higher_025 and lower_025 > higher_975:
        distance_between_intervals = lower_025 - higher_975
        return True, distance_between_intervals 
    elif higher_975 > lower_025 and higher_025 > lower_975:
        distance_between_intervals = higher_025 - lower_975
        return True, distance_between_intervals
    else:
        return False, distance_between_intervals

In [17]:
def does_not_have_enought_samples(field, threshold, data_higher, data_lower):
    """Logging for cases where the subset size is lower than the sample size

    Args:
        field (str): name of the dataframe column - with computed values for heuristics
        threshold (decimal): number on which the dataset is split between two subsets
        data_higher (Pandas Dataframe): split of dataset with values higher than the threshold
        data_lower (Pandas Dataframe): split of dataset with values lower or equal to the threshold
    """
    if write_to_log_files:
        with open("metrics_for_specific_runs.csv", "a") as file_append:
            file_append.write(f"\nlower_than_{threshold}_for_field_{field}_not_enough_samples,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},lower,-1,-1,-1,-1,-1,-1,{len(data_lower)},{len(data_higher)}")
        with open("metrics_for_specific_runs.csv", "a") as file_append:
            file_append.write(f"\nhigher_than_{threshold}_for_field_{field}_not_enough_samples,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},higher,-1,-1,-1,-1,-1,-1,{len(data_lower)},{len(data_higher)}")
        with open("average_metrics_for_runs.csv", "a") as file_append:
            file_append.write(f"\naverage_lower_than_{threshold}_for_field_{field}_not_enough_samples,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},lower,-1,-1,-1,-1,-1,-1,{len(data_lower)},{len(data_higher)}")
            file_append.write(f"\naverage_higher_than_{threshold}_for_field_{field}_not_enough_samples,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},higher,-1,-1,-1,-1,-1,-1,{len(data_lower)},{len(data_higher)}")
        with open("evaluated_metrics_for_average.csv", "a") as file_append:
            file_append.write(f"\nexact_match_not_enough_samples,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},Nan,-1,{len(data_lower)},{len(data_higher)}")
            file_append.write(f"\nf1_not_enough_samples,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},Nan,-1,{len(data_lower)},{len(data_higher)}")


In [18]:
from statistics import mean

def compute_metrics_average_split(data, field, threshold):
    """Function which calls the previous ones and provide dataset splits and logging into files

    Args:
        data (Pandas Dataframe): dataset
        field (str): column of the dataset based on which we want to split the data
        threshold (decimal): number for the split of dataset, the values lower or equal will be in one subset and values higher than the threshold will be in the other

    Returns:
        decimal, decimal: distances for both metrics, exact match and F1
    """
    data_higher, data_lower = [x for _, x in data.groupby(data[field] <= threshold)]

    if len(data_higher) < sample_size or len(data_lower) < sample_size:
        does_not_have_enought_samples(field, threshold, data_higher, data_lower)
        return -1, -1

    lower_exact_match_quantile_025 = []
    lower_exact_match_quantile_975 = []
    lower_exact_match_mean = []
    lower_f1_quantile_025 = []
    lower_f1_quantile_975 = []
    lower_f1_mean = []
    higher_exact_match_quantile_025 = []
    higher_exact_match_quantile_975 = []
    higher_exact_match_mean = []
    higher_f1_quantile_025 = []
    higher_f1_quantile_975 = []
    higher_f1_mean = []
    df_lower = []
    df_higher = []


    for i in tqdm(range(num_for_average_metrics)):
        df_lower = compute_metrics_for_bunch(data_lower)
        lower_exact_match_quantile_025.append(df_lower['exact_match'].quantile(0.025))
        lower_exact_match_quantile_975.append(df_lower['exact_match'].quantile(0.975))
        lower_exact_match_mean.append(df_lower['exact_match'].mean())
        lower_f1_quantile_025.append(df_lower['f1'].quantile(0.025))
        lower_f1_quantile_975.append(df_lower['f1'].quantile(0.975))
        lower_f1_mean.append(df_lower['f1'].mean())

        if write_to_log_files:
            with open("metrics_for_specific_runs.csv", "a") as file_append:
                file_append.write(f"\nlower_than_{threshold}_for_field_{field},{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},lower,{df_lower['exact_match'].quantile(0.025)},{df_lower['exact_match'].quantile(0.975)},{df_lower['exact_match'].mean()},{df_lower['f1'].quantile(0.025)},{df_lower['f1'].quantile(0.975)},{df_lower['f1'].mean()},{len(data_lower)},{len(data_higher)}")

        df_higher = compute_metrics_for_bunch(data_higher)
        higher_exact_match_quantile_025.append(df_higher['exact_match'].quantile(0.025))
        higher_exact_match_quantile_975.append(df_higher['exact_match'].quantile(0.975))
        higher_exact_match_mean.append(df_higher['exact_match'].mean())
        higher_f1_quantile_025.append(df_higher['f1'].quantile(0.025))
        higher_f1_quantile_975.append(df_higher['f1'].quantile(0.975))
        higher_f1_mean.append(df_higher['f1'].mean())

        if write_to_log_files:
            with open("metrics_for_specific_runs.csv", "a") as file_append:
                file_append.write(f"\nhigher_than_{threshold}_for_field_{field},{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},higher,{df_higher['exact_match'].quantile(0.025)},{df_higher['exact_match'].quantile(0.975)},{df_higher['exact_match'].mean()},{df_higher['f1'].quantile(0.025)},{df_higher['f1'].quantile(0.975)},{df_higher['f1'].mean()},{len(data_lower)},{len(data_higher)}")

    if write_to_log_files:
        with open("average_metrics_for_runs.csv", "a") as file_append:
            file_append.write(f"\naverage_lower_than_{threshold}_for_field_{field},{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},lower,{mean(lower_exact_match_quantile_025)},{mean(lower_exact_match_quantile_975)},{mean(lower_exact_match_mean)},{mean(lower_f1_quantile_025)},{mean(lower_f1_quantile_975)},{mean(lower_f1_mean)},{len(data_lower)},{len(data_higher)}")
            file_append.write(f"\naverage_higher_than_{threshold}_for_field_{field},{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},higher,{mean(higher_exact_match_quantile_025)},{mean(higher_exact_match_quantile_975)},{mean(higher_exact_match_mean)},{mean(higher_f1_quantile_025)},{mean(higher_f1_quantile_975)},{mean(higher_f1_mean)},{len(data_lower)},{len(data_higher)}")

    is_not_overlap_em, distance_em = find_the_distance_between_intervals(mean(lower_exact_match_quantile_025), mean(lower_exact_match_quantile_975), mean(higher_exact_match_quantile_025), mean(higher_exact_match_quantile_975))
    is_not_overlap_f1, distance_f1 = find_the_distance_between_intervals(mean(lower_f1_quantile_025), mean(lower_f1_quantile_975), mean(higher_f1_quantile_025), mean(higher_f1_quantile_975))

    if write_to_log_files:
        with open("evaluated_metrics_for_average.csv", "a") as file_append:
            file_append.write(f"\nexact_match,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},{is_not_overlap_em},{distance_em},{len(data_lower)},{len(data_higher)}")
            file_append.write(f"\nf1,{num_samples},{sample_size},{num_for_average_metrics},{field},{threshold},{is_not_overlap_f1},{distance_f1},{len(data_lower)},{len(data_higher)}")

    with open("metrics_with_intervals_distances_for_dataset_comparison.csv", "a") as file_append:
        file_append.write(f"\n{dataset},{field},{threshold},<{mean(lower_exact_match_quantile_025)};{mean(lower_exact_match_quantile_975)}>,{distance_em},<{mean(higher_exact_match_quantile_025)};{mean(higher_exact_match_quantile_975)}>,<{mean(lower_f1_quantile_025)};{mean(lower_f1_quantile_975)}>,{distance_f1},<{mean(higher_f1_quantile_025)};{mean(higher_f1_quantile_975)}>,{num_samples},{sample_size},{len(data_lower)},{len(data_higher)}")

    if write_to_log_files:
        for i in range(num_samples):
            with open("data_for_violin.csv", "a") as file_append:
                file_append.write(f"\n{dataset}\t{field}\t{threshold}\t<{mean(lower_exact_match_quantile_025)};{mean(lower_exact_match_quantile_975)}>\t{distance_em}\t<{mean(higher_exact_match_quantile_025)};{mean(higher_exact_match_quantile_975)}>\t<{mean(lower_f1_quantile_025)};{mean(lower_f1_quantile_975)}>\t{distance_f1}\t<{mean(higher_f1_quantile_025)};{mean(higher_f1_quantile_975)}>\t{num_samples}\t{sample_size}\t{len(data_lower)}\t{len(data_higher)}\t{df_lower['exact_match'][i]}\tlower\t{df_lower['f1'][i]}\tlower")
                file_append.write(f"\n{dataset}\t{field}\t{threshold}\t<{mean(lower_exact_match_quantile_025)};{mean(lower_exact_match_quantile_975)}>\t{distance_em}\t<{mean(higher_exact_match_quantile_025)};{mean(higher_exact_match_quantile_975)}>\t<{mean(lower_f1_quantile_025)};{mean(lower_f1_quantile_975)}>\t{distance_f1}\t<{mean(higher_f1_quantile_025)};{mean(higher_f1_quantile_975)}>\t{num_samples}\t{sample_size}\t{len(data_lower)}\t{len(data_higher)}\t{df_higher['exact_match'][i]}\thigher\t{df_higher['f1'][i]}\thigher")

    print(f"Average exact match with params: samples {sample_size} iters {num_samples} ---- are independent: {is_not_overlap_em} the distance is: {distance_em}")

    print(f"Average f1 with params: samples {sample_size} iters {num_samples} ---- are independent: {is_not_overlap_f1} the distance is: {distance_f1}")

    return distance_em, distance_f1


In [19]:
def find_longest_distance(data, field, low_bound, upp_bound):
    """Finds out the longest distance between intervals for bunch of thresholds

    Args:
        data (Pandas Dataframe): dataset
        field (str): Dataframe column
        low_bound (int): the number from which the threshold will start from
        upp_bound (int): the number for which the threshold will go -1
    """
    index_em = 0
    index_f1 = 0
    max_em_distance = 0
    max_f1_distance = 0

    distance_em = 0
    distance_f1 = 0

    for i in tqdm(range(low_bound, upp_bound, 1)):
        distance_em, distance_f1 = compute_metrics_average_split(data, field, i)
        if distance_em > max_em_distance:
            max_em_distance = distance_em
            index_em = i
        if distance_f1 > max_f1_distance:
            max_f1_distance = distance_f1
            index_f1 = i

    print(f"The biggest distance between exact match intervals was with threshold {index_em} and the distance was {max_em_distance}.")
    print(f"The biggest distance between f1 intervals was with threshold {index_f1} and the distance was {max_f1_distance}.")


### Example of run for evaluated dataset

In [ ]:
dataset = 'squad_super_all_th_2'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_all_th_2.json')

find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [21]:
# files = []
# files.append('squad_base_v3.json')
# files.append('squad_base_v4.json')
# files.append('squad_base_v5.json')
# files.append('squad_supersampled_answer_length_3.json')
# files.append('squad_supersampled_answer_length_3_v2.json')
# files.append('squad_supersampled_answer_length_3_v3.json')
# files.append('squad_supersampled_cosine_similarity_01.json')
# files.append('squad_supersampled_cosine_similarity_01_v2.json')
# files.append('squad_supersampled_cosine_similarity_01_v3.json')
# files.append('squad_supersampled_distances_7.json')
# files.append('squad_supersampled_distances_7_v2.json')
# files.append('squad_supersampled_distances_7_v3.json')
# files.append('squad_supersampled_similar_entities_0.json')
# files.append('squad_supersampled_similar_entities_0_v2.json')
# files.append('squad_supersampled_similar_entities_0_v3.json')
# files.append('squad_supersampled_similar_words_4.json')
# files.append('squad_supersampled_similar_words_4_v2.json')
# files.append('squad_supersampled_similar_words_4_v3.json')
# files.append('squad_supersampled_ans_sub_pos_1.json')
# files.append('squad_supersampled_ans_sub_pos_1_v2.json')
# files.append('squad_supersampled_ans_sub_pos_1_v3.json')
folder = './from_debiased_models/for_comparison/enhanced_'


dataset = 'squad_base'
data = pd.read_json(folder + 'squad_base_v3.json')

find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)
find_longest_distance(data, 'similar_words', 4, 5)
find_longest_distance(data, 'answer_lenght', 3, 4)
compute_metrics_average_split(data, 'cosine_similarity', 0.10)
find_longest_distance(data, 'max_sim_ents', 0, 1)
find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

dataset = 'squad_base'
data = pd.read_json(folder + 'squad_base_v4.json')

find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)
find_longest_distance(data, 'similar_words', 4, 5)
find_longest_distance(data, 'answer_lenght', 3, 4)
compute_metrics_average_split(data, 'cosine_similarity', 0.10)
find_longest_distance(data, 'max_sim_ents', 0, 1)
find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

dataset = 'squad_base'
data = pd.read_json(folder + 'squad_base_v5.json')

find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)
find_longest_distance(data, 'similar_words', 4, 5)
find_longest_distance(data, 'answer_lenght', 3, 4)
compute_metrics_average_split(data, 'cosine_similarity', 0.10)
find_longest_distance(data, 'max_sim_ents', 0, 1)
find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_answer_length_3.json')

find_longest_distance(data, 'answer_lenght', 3, 4)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_answer_length_3_v2.json')

find_longest_distance(data, 'answer_lenght', 3, 4)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_answer_length_3_v3.json')

find_longest_distance(data, 'answer_lenght', 3, 4)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_cosine_similarity_01.json')

compute_metrics_average_split(data, 'cosine_similarity', 0.10)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_cosine_similarity_01_v2.json')

compute_metrics_average_split(data, 'cosine_similarity', 0.10)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_cosine_similarity_01_v3.json')

compute_metrics_average_split(data, 'cosine_similarity', 0.10)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_distances_7.json')

find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_distances_7_v2.json')

find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_distances_7_v3.json')

find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_similar_entities_0.json')

find_longest_distance(data, 'max_sim_ents', 0, 1)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_similar_entities_0_v2.json')

find_longest_distance(data, 'max_sim_ents', 0, 1)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_similar_entities_0_v3.json')

find_longest_distance(data, 'max_sim_ents', 0, 1)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_similar_words_4.json')

find_longest_distance(data, 'similar_words', 4, 5)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_similar_words_4_v2.json')

find_longest_distance(data, 'similar_words', 4, 5)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_similar_words_4_v3.json')

find_longest_distance(data, 'similar_words', 4, 5)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_ans_sub_pos_1.json')

find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_ans_sub_pos_1_v2.json')

find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

dataset = 'squad_supersampled'
data = pd.read_json(folder + 'squad_supersampled_ans_sub_pos_1_v3.json')

find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average exact match with params: samples 800 iters 100 ---- are independent: True the distance is: 9.993750000000006
Average f1 with params: samples 800 iters 100 ---- are independent: True the distance is: 9.650621724105747
The biggest distance between exact match intervals was with threshold 7 and the distance was 9.993750000000006.
The biggest distance between f1 intervals was with threshold 7 and the distance was 9.650621724105747.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average exact match with params: samples 800 iters 100 ---- are independent: False the distance is: 0
Average f1 with params: samples 800 iters 100 ---- are independent: False the distance is: 0
The biggest distance between exact match intervals was with threshold 0 and the distance was 0.
The biggest distance between f1 intervals was with threshold 0 and the distance was 0.


In [20]:
dataset = 'squad_base'
data = pd.read_json('./enhanced_validation_squad_with_bl_predictions.json')
find_longest_distance(data, 'similar_words', 0, 10)
find_longest_distance(data, 'similar_words_stem', 0, 10)
find_longest_distance(data, 'similar_words_lem', 0, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average exact match with params: samples 800 iters 100 ---- are independent: True the distance is: 1.8687500000000057
Average f1 with params: samples 800 iters 100 ---- are independent: True the distance is: 0.9505122429934119


...

Average exact match with params: samples 800 iters 100 ---- are independent: True the distance is: 0.30937500000000284
Average f1 with params: samples 800 iters 100 ---- are independent: True the distance is: 0.5895582148752538


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average exact match with params: samples 800 iters 100 ---- are independent: True the distance is: 0.6843750000000028
Average f1 with params: samples 800 iters 100 ---- are independent: True the distance is: 0.7078659538193222
The biggest distance between exact match intervals was with threshold 9 and the distance was 0.6843750000000028.
The biggest distance between f1 intervals was with threshold 9 and the distance was 0.7078659538193222.


In [28]:
find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)
find_longest_distance(data, 'similar_words', 4, 5)
find_longest_distance(data, 'answer_lenght', 3, 4)
compute_metrics_average_split(data, 'cosine_similarity', 0.10)
find_longest_distance(data, 'kth_sentence', 0, 1)
find_longest_distance(data, 'max_sim_ents', 0, 1)
find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 1, 2)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average exact match with params: samples 800 iters 100 ---- are independent: True the distance is: 8.803125000000009
Average f1 with params: samples 800 iters 100 ---- are independent: True the distance is: 8.320067159352774
The biggest distance between exact match intervals was with threshold 7 and the distance was 8.803125000000009.
The biggest distance between f1 intervals was with threshold 7 and the distance was 8.320067159352774.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Average exact match with params: samples 800 iters 100 ---- are independent: True the distance is: 0.8031250000000085
Average f1 with params: samples 800 iters 100 ---- are independent: True the distance is: 0.39578747090899924
The biggest distance between exact match intervals was with threshold 1 and the distance was 0.8031250000000085.
The biggest distance between f1 intervals was with threshold 1 and the distance was 0.39578747090899924.


In [ ]:
dataset = 'squad_distances_v2'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_distances_7_v2.json')
for i in range(3): #one more run
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
# enhanced_squad_base_v2

dataset = 'squad_base_v2'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_base_v2.json')
for i in range(3): #one more run
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
find_longest_distance(data[data.distances >= 0], 'distances', 7, 8)
find_longest_distance(data, 'similar_words', 4, 5)
find_longest_distance(data, 'answer_lenght', 3, 4)

In [ ]:
sample_size = 450
find_longest_distance(data, 'kth_sentence', 0, 5)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.10)
compute_metrics_average_split(data, 'cosine_similarity', 0.20)
compute_metrics_average_split(data, 'cosine_similarity', 0.30)
compute_metrics_average_split(data, 'cosine_similarity', 0.40)
compute_metrics_average_split(data, 'cosine_similarity', 0.50)
compute_metrics_average_split(data, 'cosine_similarity', 0.60)

In [ ]:
sample_size = 450
compute_metrics_average_split(data, 'cosine_similarity', 0.10)

In [ ]:
sample_size = 800

In [ ]:
find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
find_longest_distance(data, 'max_sim_ents', 0, 5)

In [ ]:
find_longest_distance(data, 'question_length', 5, 16)

In [ ]:
dataset = 'squad_super_all_th_2'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_all_th_2.json')
for i in range(3): #one more run
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
dataset = 'squad_super_all_v1'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_all_v1.json')
for i in range(3):
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
dataset = 'squad_super_all_v2'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_all_v2.json')
for i in range(3):
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
dataset = 'squad_super_all_dd_ns'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_all_dd_ns.json')
for i in range(3):
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
dataset = 'squad_base'
data = data = pd.read_json('valid_pred_labeled_with_added_from_func.json')
for i in range(3):
    find_longest_distance(data[data.distances >= 0], 'distances', 2, 9)
    find_longest_distance(data, 'similar_words', 3, 9)
    find_longest_distance(data, 'answer_lenght', 1, 6)
    compute_metrics_average_split(data, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data, 'cosine_similarity', 0.60)
    find_longest_distance(data, 'max_sim_ents', 0, 5)
    find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
dataset = 'squad_super_cosine_similarity_01'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_cosine_similarity_01.json')
for i in range(3):
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.10)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.20)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.30)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.40)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.50)
    compute_metrics_average_split(data_supersampled_model, 'cosine_similarity', 0.60)

In [ ]:
dataset = 'squad_super_max_sim_ents_0'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_similar_entities_0.json')
for i in range(3):
    find_longest_distance(data_supersampled_model, 'max_sim_ents', 0, 5)

In [ ]:
dataset = 'squad_super_ans_sub_pos_1'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_ans_sub_pos_1.json')
for i in range(3):
    find_longest_distance(data_supersampled_model[data_supersampled_model.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
dataset = 'squad_base'
data = pd.read_json('valid_pred_labeled_with_added_from_func.json')
for i in range(4):
    find_longest_distance(data[data.distances >= 0], 'distances', 2, 9)

In [ ]:
dataset = 'squad_super_distances_7'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_distances_7.json')
for i in range(4):
    find_longest_distance(data_supersampled_model[data_supersampled_model.distances >= 0], 'distances', 2, 9)

In [ ]:
dataset = 'squad_base'
data = pd.read_json('valid_pred_labeled_with_added_from_func.json')
for i in range(5):
    find_longest_distance(data, 'similar_words', 3, 9)

In [ ]:
dataset = 'squad_super_similar_words_4'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_similar_words_4.json')
for i in range(5):
    find_longest_distance(data_supersampled_model, 'similar_words', 3, 9)

In [ ]:
dataset = 'squad_base'
data = pd.read_json('valid_pred_labeled_with_added_from_func.json')
for i in range(5):
    find_longest_distance(data, 'answer_lenght', 1, 6)

In [ ]:
dataset = 'squad_super_answer_length_3'
data_supersampled_model = pd.read_json('./from_debiased_models/enhanced_squad_supersampled_answer_length_3.json')
for i in range(5):
    find_longest_distance(data_supersampled_model, 'answer_lenght', 1, 6)

In [ ]:
# find_longest_distance(data[data.distances_new >= 0], 'distances_new', 2, 7)
find_longest_distance(data[data.distances_new >= 0], 'distances_new', 7, 9)

In [ ]:
find_longest_distance(data[data.distances >= 0], 'distances', 2, 7)

In [ ]:
find_longest_distance(data, 'kth_sentence_new', 0, 5)

In [ ]:
# data = data[data.answer_subject_positions >= 0]
find_longest_distance(data[data.answer_subject_positions >= 0], 'answer_subject_positions', 0, 2)

In [ ]:
data = data[data.answer_subject_positions >= 0]
find_longest_distance(data, 'answer_subject_positions', 0, 2)

In [ ]:
data = data[data.answer_subject_positions >= 0]
find_longest_distance(data, 'answer_subject_positions', 0, 2)

In [ ]:
# data = data[data.subject_in_context_count >= 0]
find_longest_distance(data[data.subject_in_context_count >= 0], 'subject_in_context_count', 0, 3)

In [ ]:
find_longest_distance(data, 'max_sim_ents', 0, 5)

In [ ]:
find_longest_distance(data, 'max_sim_ents', 0, 5)

In [ ]:
find_longest_distance(data, 'subject_in_context_count', 0, 2)

In [ ]:
find_longest_distance(data, 'answer_subject_positions', 0, 2)

In [ ]:
find_longest_distance(data, 'answer_subject_positions', 0, 5)

In [ ]:
find_longest_distance(data, 'distances_new', 2, 7)

In [ ]:
find_longest_distance(data, 'distances', 2, 7)

In [ ]:
data.loc[data['distances'] == -1, 'distances'] = 1000
data['distances'].value_counts()

In [ ]:
find_longest_distance(data, 'distances', 2, 7)

In [ ]:
find_longest_distance(data, 'answer_lenght', 1, 5)

In [ ]:
for size in range(200, 1100, 100):
    sample_size = size
    for iters in range(100, 600, 100):
        num_samples = iters
        find_longest_distance(data, 'distances', 2, 7)

In [ ]:
for size in range(900, 1100, 100):
    sample_size = size
    for iters in range(100, 600, 100):
        num_samples = iters
        find_longest_distance(data, 'distances', 2, 7)

In [ ]:
for size in range(200, 1100, 100):
    sample_size = size
    for iters in range(100, 600, 100):
        num_samples = iters
        find_longest_distance(data, 'similar_words', 3, 10)

In [ ]:
for size in range(200, 1100, 200):
    sample_size = size
    for iters in range(100, 600, 200):
        num_samples = iters
        find_longest_distance(data, 'kth_sentence', 0, 5)

In [ ]:
for size in range(200, 1100, 200):
    sample_size = size
    for iters in range(100, 600, 200):
        num_samples = iters
        compute_metrics_average_split(data, 'cosine_similarity', 0.10)
        compute_metrics_average_split(data, 'cosine_similarity', 0.20)
        compute_metrics_average_split(data, 'cosine_similarity', 0.30)
        compute_metrics_average_split(data, 'cosine_similarity', 0.40)
        compute_metrics_average_split(data, 'cosine_similarity', 0.50)
        compute_metrics_average_split(data, 'cosine_similarity', 0.60)

In [ ]:
find_longest_distance(data, 'distances', 2, 7)

In [ ]:
find_longest_distance(data, 'similar_words', 3, 10)

In [ ]:
find_longest_distance(data, 'similar_words', 2, 6)

In [ ]:
find_longest_distance(data, 'kth_sentence', 0, 5) #zvacsit interval, skusit pre vacsie k
# find_longest_distance(data, 'kth_sentence', 5, 8) #zvacsit interval, skusit pre vacsie k
# find_longest_distance(data, 'kth_sentence', 1, 5) #zvacsit interval, skusit pre vacsie k

In [ ]:
num_samples = 200
sample_size = 1000

find_longest_distance(data, 'kth_sentence', 4, 6)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.10)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.15)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.20)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.25)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.30)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.35)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.40)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.45)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.50)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.55)

In [ ]:
compute_metrics_average_split(data, 'distances', 5)

In [ ]:
compute_metrics_average_split(data, 'distances', 6)

In [ ]:
index_em = 0
index_f1 = 0
max_em_distance = 0
max_f1_distance = 0

distance_em = 0
distance_f1 = 0

for i in range(3, 6, 1):
    distance_em, distance_f1 = compute_metrics_average_split(data, 'similar_words', i)
    if distance_em > max_em_distance:
        max_em_distance = distance_em
        index_em = i
    if distance_f1 > max_f1_distance:
        max_f1_distance = distance_f1
        index_f1 = i

print(f"The biggest distance between exact match intervals was with threshold {index_em} and the distance was {max_em_distance}.")
print(f"The biggest distance between f1 intervals was with threshold {index_f1} and the distance was {max_f1_distance}.")

In [ ]:
compute_metrics_average_split(data, 'similar_words', 2)

In [ ]:
compute_metrics_average_split(data, 'distances', 3)

In [ ]:
compute_metrics_average_split(data, 'similar_words', 4)

In [ ]:
compute_metrics_average_split(data, 'kth_sentence', 3)

In [ ]:
compute_metrics_average_split(data, 'kth_sentence', 2)

In [ ]:
compute_metrics_average_split(data, 'kth_sentence', 1)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.35)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.25)

In [ ]:
compute_metrics_average_split(data, 'cosine_similarity', 0.45)

### Older segments of code, now used in functions above

In [ ]:
data_distances = data[data.distances >= 0]
data_higher, data_lower = [x for _, x in data.groupby(data_distances['distances'] <= 3)]

In [ ]:
exact_list_lower = []
f1_list_lower = []

for i in tqdm(range(num_samples)):
    df = data_lower.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_lower.append(metrics1['exact_match'])
    f1_list_lower.append(metrics1['f1'])

In [ ]:
d_lower = {'exact_match': exact_list_lower, 'f1': f1_list_lower}
d_lower
lower_than_4 = pd.DataFrame(d_lower)
lower_than_4

In [ ]:
exact_list_higher = []
f1_list_higher = []

for i in tqdm(range(num_samples)):
    df = data_higher.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_higher.append(metrics1['exact_match'])
    f1_list_higher.append(metrics1['f1'])

In [ ]:
d_higher = {'exact_match': exact_list_higher, 'f1': f1_list_higher}
d_higher
higher_than_4 = pd.DataFrame(d_higher)
higher_than_4

### Quantiles

In [ ]:
lower_than_4.quantile([0.025, 0.975])

In [ ]:
higher_than_4.quantile([0.025, 0.975])

In [ ]:
lower_than_4.describe()

In [ ]:
higher_than_4.describe()

### Spliting of the data

#### Based on count of the similar words between question and context - lower or equal to 4 and higher than 4

In [ ]:
data_similar_words_higher, data_similar_words_lower = [x for _, x in data.groupby(data['similar_words'] <= 4)]
print('Higher count data len: ', len(data_similar_words_higher))
print('Lower count data len: ', len(data_similar_words_lower))

### Computation of metrics for samples

In [ ]:
exact_list_similar_words_lower = []
f1_list_similar_words_lower = []

for i in tqdm(range(num_samples)):
    df = data_similar_words_lower.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_similar_words_lower.append(metrics1['exact_match'])
    f1_list_similar_words_lower.append(metrics1['f1'])

In [ ]:
d_similar_words_lower = {'exact_match': exact_list_similar_words_lower, 'f1': f1_list_similar_words_lower}
d_similar_words_lower
lower_similar_words_than_4 = pd.DataFrame(d_similar_words_lower)
lower_similar_words_than_4

In [ ]:
exact_list_similar_words_higher = []
f1_list_similar_words_higher = []

for i in tqdm(range(num_samples)):
    df = data_similar_words_higher.sample(n=sample_size)
    sample = Dataset.from_pandas(df)
    metrics1 = compute_metrics_for_sample(sample)
    exact_list_similar_words_higher.append(metrics1['exact_match'])
    f1_list_similar_words_higher.append(metrics1['f1'])

In [ ]:
d_similar_words_higher = {'exact_match': exact_list_similar_words_higher, 'f1': f1_list_similar_words_higher}
d_similar_words_higher
higher_similar_words_than_4 = pd.DataFrame(d_similar_words_higher)
higher_similar_words_than_4

### Quantiles

In [ ]:
lower_similar_words_than_4.quantile([0.025, 0.975])

In [ ]:
higher_similar_words_than_4.quantile([0.025, 0.975])

In [ ]:
lower_similar_words_than_4.describe()

In [ ]:
higher_similar_words_than_4.describe()